In [8]:
import numpy as np
import pandas as pd

In [9]:
data = pd.read_csv('../data/real_estate.csv')
data

,id,price,address,coordinates,region,subway,rooms,footage,floor,features,residential,neighborhood,description,detail,attributes
0,11312688,18000,"Саксаганського вул., 12б","30.51733208,50.43645859","Київ,Печерський р-н",Палац Спорту,1 кімната,32 / 25 / 7 м²,поверх 5 з 7,"['Стара цегла', 'Роздільне', 'Задовільний стан']",NaN,"['Спортивна площа', 'Стадіон Олімпійський', 'Ч...","Будинок - Стара цегла, в квартирі 1 кімната. П...","\nКиїв, Саксаганського вул., 12б, Печерський р-н","['Душова кабіна', 'Лічильники', 'Мікрохвильовк..."
1,11305787,58500,"Дніпровська наб., 20Б","30.61351013,50.3984375","Київ,Дарницький р-н",Осокорки,3 кімнати,71 / 30 / 25 м²,поверх 25 з 27,['Дизайнерський ремонт'],ЖК Seven,['ТЦ River Mall'],Оренда квартири\n\nВашій увазі пропонується ро...,"В квартирі 3 кімнати. В квартирі є Телевізор, ...","['Кондиціонер', 'Мікрохвильовка', 'Пральна маш..."
2,11313827,16000,"Сікорського Ігоря вул. (Танкова), 4Б","30.42817307,50.4651947","Київ,Шевченківський р-н",Берестейська,1 кімната,45 / 18 / 14 м²,поверх 25 з 26,['Чудовий стан'],NaN,['Парк Нивки'],"У квартирі виконаний євроремонт, необхідні меб...",В квартирі 1 кімната. Гаряча вода - централізо...,"['Душова кабіна', 'Лічильники', 'Кондиціонер',..."
3,11187262,272900,"Новоселицька вул., 10","30.55553436,50.40839767","Київ,Печерський р-н",Видубичі,4 кімнати,200 / 120 / 40 м²,поверх 7 з 9,"['Українська цегла', 'Роздільне', 'Дизайнерськ...",ЖК Renaissance Residence,['Національний ботанічний сад ім. М. М. Гришка...,ОРЕНДА завжди є світло!! Автономне опалення!\n...,"Будинок - Українська цегла, в квартирі 4 кімна...","['Посудомийна машина', 'Душова кабіна', 'Супут..."
4,11285717,31000,"Менделєєва вул., 19","30.53597069,50.41062927","Київ,Печерський р-н",Либідська,2 кімнати,38 / 15 / 18 м²,поверх 4 з 6,"['Бетонно монолітний', 'Роздільне', 'Перша зда...",ЖК ONYX sweet home,"['Чорна гора', 'Новопечерські липки', 'ТЦ Ocea...",Перша здача.\nБез Комісії!\nЯкщо Ви бачите це ...,"Будинок - Бетонно монолітний, в квартирі 2 кім...","['Посудомийна машина', 'Душова кабіна', 'Лічил..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17191,11391023,48700,"Інститутська вул., 15/5","30.53435326,50.44621658","Київ,Печерський р-н",Арсенальна,3 кімнати,90 / 50 / 25 м²,поверх 4 з 8,"['Сталінка', 'Суміжно-роздільна', 'Дизайнерськ...",NaN,"['Маріїнський парк', 'Офіс Президента', 'Липки...",Шукаєте для себе красиву квартиру в історичній...,"Будинок - Сталінка, в квартирі 3 кімнати. План...","['Душова кабіна', 'Кондиціонер', 'Посуд', 'Мік..."
17192,11410579,46600,"Нижній Вал вул., 35","30.52472496,50.45049286","Київ,Шевченківський р-н",Майдан Незалежності,2 кімнати,67 / 30 / 30 м²,поверх 3 з 3,"['Українська цегла', 'Суміжно-роздільна', 'Євр...",NaN,"['Майдан Незалежності', 'Європейська площа', '...",Актуально! Можливі перегляди!\n\nПропонуємо в ...,"Будинок - Українська цегла, в квартирі 2 кімна...","['Посудомийна машина', 'Кондиціонер', 'Мікрохв..."
17193,11423818,35000,"Туманяна Ованеса вул., 15а","30.60375214,50.4439888","Київ,Дніпровський р-н",Лівобережна,3 кімнати,125 / 65 / 16 м²,поверх 19 з 27,"['Роздільне', 'Дизайнерський ремонт']",ЖК Лазурний блюз,"['Русанівка', 'Проспект Соборності']",Оренда 3 кімнатної квартири в класному будинку...,В квартирі 3 кімнати. Планування кімнат Розділ...,[]
17194,11420073,109500,"Воздвиженська вул., 29","30.51169968,50.46016312","Київ,Подільський р-н",Контрактова площа,3 кімнати,127 / 64 / 18 м²,поверх 2 з 4,['Дизайнерський ремонт'],NaN,"['Воздвиженка', 'Андріївський Узвіз', 'Пейзажн...","БЕЗ КОМИССИИ! Уникальная квартира, S - 126,8/6...",В квартирі 3 кімнати. Загальний стан квартири ...,[]


### Drop all possible duplicates

In [10]:
def drop_all_duplicates(df):
    data = df.copy()

    data.drop_duplicates(keep='last', inplace=True, ignore_index=True)

    data.replace('[]', np.nan, inplace=True)

    features = ['attributes', 'coordinates', 'subway', 'residential', 'neighborhood']

    data_dupl = (data[
        data.duplicated(subset=['address', 'rooms', 'footage', 'floor'], keep=False)]
        .sort_values(by=['address']))

    for feat in features:
        data_dupl[feat] = data_dupl[feat].fillna(
            data_dupl['address'].map(data_dupl
                                         .dropna(subset=[feat])
                                         .drop_duplicates(subset=['address'], keep='last')
                                         .set_index('address')
                                         [feat]
                                         )
        )
        data[feat] = data[feat].fillna(data_dupl[feat])

    data.drop_duplicates(subset=['address', 'rooms', 'footage', 'floor'],
                         keep='last',
                         inplace=True,
                         ignore_index=True)

    data.drop_duplicates(subset=['id'],
                         keep='last',
                         inplace=True,
                         ignore_index=True)

    return data

In [11]:
df = drop_all_duplicates(data)
df.shape

(9879, 15)

### Clean data

In [12]:
def prepare_df(df):
    data = df.copy()
    data.loc[data['rooms'] == 'Вільне планування', 'rooms'] = '1 кімната'
    data['num_rooms'] = data['rooms'].str.split(' ').str[0].astype('int8')
    data[['full_area',
          'living_area',
          'kitchen_area']] = (data['footage']
                              .str.rsplit(' ', n=1)
                              .str[0]
                              .str.split(' / ', expand=True)).astype('float')

    data[['floor', 'num_storeys']] = (data['floor']
                                      .str.split(' ', n=1)
                                      .str[1]
                                      .str.strip()
                                      .str.split(' з ', expand=True)).astype('int')

    del data['rooms']
    del data['footage']

    return data


In [13]:
df_clean = prepare_df(df)
df_clean.shape

(9879, 18)

In [14]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9879 entries, 0 to 9878
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            9879 non-null   int64  
 1   price         9879 non-null   int64  
 2   address       9879 non-null   object 
 3   coordinates   9784 non-null   object 
 4   region        9879 non-null   object 
 5   subway        7995 non-null   object 
 6   floor         9879 non-null   int32  
 7   features      9879 non-null   object 
 8   residential   4832 non-null   object 
 9   neighborhood  8956 non-null   object 
 10  description   9879 non-null   object 
 11  detail        9878 non-null   object 
 12  attributes    6474 non-null   object 
 13  num_rooms     9879 non-null   int8   
 14  full_area     9879 non-null   float64
 15  living_area   9879 non-null   float64
 16  kitchen_area  9879 non-null   float64
 17  num_storeys   9879 non-null   int32  
dtypes: float64(3), int32(2), int

In [15]:
df_clean.to_csv('../data/real_estate_clean.csv', index=False)